In [1]:
import os
import ast
import openai
import time
import json

In [1]:
# openai.organization = ""  #My OpenAI account
# openai.api_key = "" #My Key
# openai.Model.list()

In [15]:
with open('sentence_selection/selected_sentence_human2.json', 'r') as f:
    data = json.load(f)
print(len(data))

600


In [16]:
# Call GPT3.5 Model 
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo-16k", 
                                 temperature=0, 
                                 max_tokens=1024):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=temperature, # this is the degree of randomness of the model's output
            max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
        )
        return response.choices[0].message["content"]
    except openai.error.RateLimitError:
        return None

In [17]:
def retry_with_exponential_backoff(input_text, max_retries=5):
    retry_count = 0
    wait_time = 1  # initial wait time in seconds

    while retry_count < max_retries:
        response = get_completion_from_messages(input_text)
        if response is not None:
            return response

        time.sleep(wait_time)
        wait_time *= 2  # double the wait time for next retry
        retry_count += 1

    raise Exception("Max retries reached")

In [18]:
delimiter_1 = "#"
delimiter_2 = "##"

system_message = f"""
You are a Fact Checker.\
The user will give you a news paragraph and several evidence sentences.\
The evidence sentence may contain information that supports or refutes the news paragraph’s central claim. \
Determine whether the news story is factual or not based on the evidence sentences.\ 
Pick phrases or sentences from both the news story and the evidence to support your conclusion.\
The user will put the news paragraph behind {delimiter_1} and the relevant sentence behind {delimiter_2}. 
"""


In [54]:
i = 590

In [55]:
for item in data[590:]:
    print(f"Predicting {i}")
    
    news = item['abstractive']
    #print(summary)
    evidence_sentence = item['selected_sentence']
    input_evidence_sentence = ""
    for sentence in evidence_sentence:
        input_evidence_sentence = input_evidence_sentence + sentence
   

    user_message = f"""
    I will give you one news paragraph {delimiter_1}{news} and several relevant sentences {delimiter_2}{evidence_sentence}.\
    Please help me determine if these sentences support or refute the news point of view. \
    Finally, please answer using one word'refute' or 'support' and give reasons.\
    Please provides the final output in JSON format containing the following two keys: prediction and reason.
    """

    messages =  [  
                {'role':'system', 'content': system_message},    
                {'role':'user', 'content': user_message},  
                ]
    response = retry_with_exponential_backoff(messages)
    print(response)        
    response_json = json.loads(response)
    #processed_response = response.split(delimiter_1)[1:]
    item['prediction'] = response_json['prediction']
    item['reason'] = response_json['reason']
    #print(response)
    
    time.sleep(2)
    i += 1

Predicting 590
{"prediction": "refute", "reason": "The evidence sentences provide information about rare blood clots with low platelets being reported in a small number of people who received the AstraZeneca vaccine. This contradicts the news paragraph's claim of embalmers finding unusually long and fibrous blood clots in bodies of vaccinated individuals. The evidence sentences specifically mention that the reported blood clots are rare and different from what the embalmers are claiming to have found."}
Predicting 591
{"prediction": "refute", "reason": "The evidence sentences provided do not directly support or refute the news paragraph. The evidence sentences discuss suicide as a general public health concern, but do not provide any specific information about Katarina Pavelek's case or the link between her suicide and the Covid-19 booster shot."}
Predicting 592
{"prediction": "refute", "reason": "The evidence sentences do not provide any credible sources or data to support the claim m

In [56]:
FILE_PATH = 'prediction/prediction_human2_zeroshot.json'
with open(FILE_PATH, 'w') as output_file:
	print(json.dumps(data, indent=2), file=output_file)